### Retrieving Normal sinus rythms, atrial fibrillation rythms and atrial flutter rythms
### From Physionet Databases AFDB, LTAFDB, MITDB and NSRDB

In [5]:
import wfdb
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# FOR AFDB

In [7]:
records = []
properties = []
annot = []
AnnSymb = []
AnnSamp = []
AnnRhythm = []
Rpeak_Samp = []
Rpeak_Symb = []


# Choose the local path were the Physionet data is stored
# 'D:/SCHOOL/9_Semester (40 ECTS SPECIALE)/40 ECTS SPECIALE/code/Master_work/data/atrial_flutter_detection/data/AFDB/*.dat'
for f in glob.glob('data/AFDB/*.dat'): ##### change the path to own directory.       
                                        

    sig, fields = wfdb.rdsamp(f[:-4], channels=[1])  #### In this function, pass 
                                                     #### "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    ann = wfdb.rdann(f[:-4], 'atr')
    QRS = wfdb.rdann(f[:-4], 'qrs')
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
    QRS_Symb = pd.Series(QRS.symbol)
    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records.append(sig)
    properties.append(fields)
    annot.append(ann)
    AnnSymb.append(Symb)
    AnnSamp.append(Samp)
    Rpeak_Symb.append(QRS_Symb)
    Rpeak_Samp.append(QRS_Samp)
    AnnRhythm.append(Rhythm)
    
AnnSymb = pd.Series(AnnSymb).values
AnnSamp = pd.Series(AnnSamp).values

In [13]:
labeled_Rpeaks = []
appended_data = []
k=1
for i in range(23): ## 23 records
    for j in range(len(AnnSamp[i])-1): ## AnnSamp or AnnRhythm -- both same dimensions
        df = pd.DataFrame(Rpeak_Samp[i][(Rpeak_Samp[i] > AnnSamp[i][j]) & (Rpeak_Samp[i] < AnnSamp[i][j+1])])
        df['Label'] = AnnRhythm[i][j]
        df['ids'] = k
        appended_data.append(df)
    k+=1    

labeled_Rpeaks = pd.concat(appended_data)

labeled_Rpeaks_N_AFIB_AFL = labeled_Rpeaks.loc[(labeled_Rpeaks.Label == '(N') | (labeled_Rpeaks.Label == '(AFIB') | (labeled_Rpeaks.Label == '(AFL'), :]
labeled_Rpeaks_N_AFIB_AFL['Label'] = labeled_Rpeaks_N_AFIB_AFL['Label'].map({'(N':0 ,'(AFIB':1, '(AFL':2})
labeled_Rpeaks_N_AFIB_AFL['Label'] = labeled_Rpeaks_N_AFIB_AFL['Label'].astype(int)
labeled_Rpeaks_N_AFIB_AFL.rename(columns={0: 'Rpeaks'},inplace=True, errors='raise')

C:\Users\masud\AppData\Local\Temp\ipykernel_15172\144449108.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_Rpeaks_N_AFIB_AFL['Label'] = labeled_Rpeaks_N_AFIB_AFL['Label'].map({'(N':0 ,'(AFIB':1, '(AFL':2})
C:\Users\masud\AppData\Local\Temp\ipykernel_15172\144449108.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_Rpeaks_N_AFIB_AFL['Label'] = labeled_Rpeaks_N_AFIB_AFL['Label'].astype(int)
C:\Users\masud\AppData\Local\Temp\ipykernel_15172\144449108.py:17: SettingWithCopyWarning: 
A v

In [14]:
labeled_Rpeaks_N_AFIB_AFL

,Rpeaks,Label,ids
0,61,0,1
1,200,0,1
2,358,0,1
3,584,0,1
4,729,0,1
...,...,...,...
15373,2849597,0,23
15374,2849785,0,23
15375,2849975,0,23
15376,2850083,0,23


### Checking for wrong rythms in db

In [21]:
values_to_keep = [0,1,2] ### 713954 (total) - 708567 (NOT AFL) = 5387 AFL labels;   = 253862 AFIB labels;   = 454705 N labels
bool_result = labeled_Rpeaks_N_AFIB_AFL['Label'].isin(values_to_keep)
wrong_rythms = [i for i in bool_result if i is False]
len(wrong_rythms)

0

# FOR LTAFDB

In [23]:
records = []
properties = []
annot = []
AnnSymb = []
AnnSamp = []
AnnRhythm = []
Rpeak_Samp = []
Rpeak_Symb = []


# Choose the local path were the Physionet data is stored
# 'D:/SCHOOL/9_Semester (40 ECTS SPECIALE)/40 ECTS SPECIALE/code/Master_work/data/atrial_flutter_detection/data/AFDB/*.dat'
for f in glob.glob('data/LTAFDB/*.dat'): ##### change the path to own directory.       
                                        

    sig, fields = wfdb.rdsamp(f[:-4], channels=[1])  #### In this function, pass 
                                                     #### "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    ann = wfdb.rdann(f[:-4], 'atr')
    QRS = wfdb.rdann(f[:-4], 'qrs')
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
    QRS_Symb = pd.Series(QRS.symbol)
    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records.append(sig)
    properties.append(fields)
    annot.append(ann)
    AnnSymb.append(Symb)
    AnnSamp.append(Samp)
    Rpeak_Symb.append(QRS_Symb)
    Rpeak_Samp.append(QRS_Samp)
    AnnRhythm.append(Rhythm)
    
AnnSymb = pd.Series(AnnSymb).values
AnnSamp = pd.Series(AnnSamp).values

In [ ]:
labeled_Rpeaks_LTAFDB = []
appended_data_LTAFDB = []
k=1
for i in range(23): ## 23 records
    for j in range(len(AnnSamp[i])-1): ## AnnSamp or AnnRhythm -- both same dimensions
        df = pd.DataFrame(Rpeak_Samp[i][(Rpeak_Samp[i] > AnnSamp[i][j]) & (Rpeak_Samp[i] < AnnSamp[i][j+1])])
        df['Label'] = AnnRhythm[i][j]
        df['ids'] = k
        appended_data_LTAFDB.append(df)
    k+=1    

labeled_Rpeaks_LTAFDB = pd.concat(appended_data_LTAFDB)

labeled_Rpeaks_LTAFDB_final = labeled_Rpeaks_LTAFDB.loc[(labeled_Rpeaks_LTAFDB.Label == '(N') | (labeled_Rpeaks_LTAFDB.Label == '(AFIB') | (labeled_Rpeaks_LTAFDB.Label == '(AFL'), :]
labeled_Rpeaks_LTAFDB_final['Label'] = labeled_Rpeaks_LTAFDB_final['Label'].map({'(N':0 ,'(AFIB':1, '(AFL':2})
labeled_Rpeaks_LTAFDB['Label'] = labeled_Rpeaks_LTAFDB_final['Label'].astype(int)
labeled_Rpeaks_LTAFDB_final.rename(columns={0: 'Rpeaks'},inplace=True, errors='raise')

In [ ]:
labeled_Rpeaks_LTAFDB_final

# FOR MITDB

In [ ]:
records = []
properties = []
annot = []
AnnSymb = []
AnnSamp = []
AnnRhythm = []
Rpeak_Samp = []
Rpeak_Symb = []


# Choose the local path were the Physionet data is stored
# 'D:/SCHOOL/9_Semester (40 ECTS SPECIALE)/40 ECTS SPECIALE/code/Master_work/data/atrial_flutter_detection/data/AFDB/*.dat'
for f in glob.glob('data/MITDB/*.dat'): ##### change the path to own directory.       
                                        

    sig, fields = wfdb.rdsamp(f[:-4], channels=[1])  #### In this function, pass 
                                                     #### "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    ann = wfdb.rdann(f[:-4], 'atr')
    QRS = wfdb.rdann(f[:-4], 'qrs')
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
    QRS_Symb = pd.Series(QRS.symbol)
    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records.append(sig)
    properties.append(fields)
    annot.append(ann)
    AnnSymb.append(Symb)
    AnnSamp.append(Samp)
    Rpeak_Symb.append(QRS_Symb)
    Rpeak_Samp.append(QRS_Samp)
    AnnRhythm.append(Rhythm)
    
AnnSymb = pd.Series(AnnSymb).values
AnnSamp = pd.Series(AnnSamp).values

In [ ]:
labeled_Rpeaks_MITDB = []
appended_data_MITDB = []
k=1
for i in range(23): ## 23 records
    for j in range(len(AnnSamp[i])-1): ## AnnSamp or AnnRhythm -- both same dimensions
        df = pd.DataFrame(Rpeak_Samp[i][(Rpeak_Samp[i] > AnnSamp[i][j]) & (Rpeak_Samp[i] < AnnSamp[i][j+1])])
        df['Label'] = AnnRhythm[i][j]
        df['ids'] = k
        appended_data_MITDB.append(df)
    k+=1    

labeled_Rpeaks_MITDB = pd.concat(appended_data_MITDB)

labeled_Rpeaks_MITDB_final = labeled_Rpeaks_MITDB.loc[(labeled_Rpeaks_MITDB.Label == '(N') | (labeled_Rpeaks_MITDB.Label == '(AFIB') | (labeled_Rpeaks_MITDB.Label == '(AFL'), :]
labeled_Rpeaks_MITDB_final['Label'] = labeled_Rpeaks_MITDB_final['Label'].map({'(N':0 ,'(AFIB':1, '(AFL':2})
labeled_Rpeaks_MITDB['Label'] = labeled_Rpeaks_MITDB_final['Label'].astype(int)
labeled_Rpeaks_MITDB_final.rename(columns={0: 'Rpeaks'},inplace=True, errors='raise')

# FOR NSRDB

In [ ]:
records = []
properties = []
annot = []
AnnSymb = []
AnnSamp = []
AnnRhythm = []
Rpeak_Samp = []
Rpeak_Symb = []


# Choose the local path were the Physionet data is stored
# 'D:/SCHOOL/9_Semester (40 ECTS SPECIALE)/40 ECTS SPECIALE/code/Master_work/data/atrial_flutter_detection/data/AFDB/*.dat'
for f in glob.glob('data/NSRDB/*.dat'): ##### change the path to own directory.       
                                        

    sig, fields = wfdb.rdsamp(f[:-4], channels=[1])  #### In this function, pass 
                                                     #### "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    ann = wfdb.rdann(f[:-4], 'atr')
    QRS = wfdb.rdann(f[:-4], 'qrs')
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
    QRS_Symb = pd.Series(QRS.symbol)
    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records.append(sig)
    properties.append(fields)
    annot.append(ann)
    AnnSymb.append(Symb)
    AnnSamp.append(Samp)
    Rpeak_Symb.append(QRS_Symb)
    Rpeak_Samp.append(QRS_Samp)
    AnnRhythm.append(Rhythm)
    
AnnSymb = pd.Series(AnnSymb).values
AnnSamp = pd.Series(AnnSamp).values

In [ ]:
labeled_Rpeaks_NSRDB = []
appended_data_NSRDB = []
k=1
for i in range(23): ## 23 records
    for j in range(len(AnnSamp[i])-1): ## AnnSamp or AnnRhythm -- both same dimensions
        df = pd.DataFrame(Rpeak_Samp[i][(Rpeak_Samp[i] > AnnSamp[i][j]) & (Rpeak_Samp[i] < AnnSamp[i][j+1])])
        df['Label'] = AnnRhythm[i][j]
        df['ids'] = k
        appended_data_NSRDB.append(df)
    k+=1    

labeled_Rpeaks_NSRDB = pd.concat(appended_data_NSRDB)

labeled_Rpeaks_NSRDB_final = labeled_Rpeaks_NSRDB.loc[(labeled_Rpeaks_NSRDB.Label == '(N') | (labeled_Rpeaks_NSRDB.Label == '(AFIB') | (labeled_Rpeaks_NSRDB.Label == '(AFL'), :]
labeled_Rpeaks_NSRDB_final['Label'] = labeled_Rpeaks_NSRDB_final['Label'].map({'(N':0 ,'(AFIB':1, '(AFL':2})
labeled_Rpeaks_NSRDB['Label'] = labeled_Rpeaks_NSRDB_final['Label'].astype(int)
labeled_Rpeaks_NSRDB_final.rename(columns={0: 'Rpeaks'},inplace=True, errors='raise')